<a href="https://colab.research.google.com/github/gtmray/Covid-19-Nepal/blob/master/covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
try:
  import requests
  from bs4 import BeautifulSoup
  import pandas as pd
  import numpy as np
except:
  print('IMPORT ERROR!! TYPE !pip install "package-name" to install')

In [38]:
url = 'https://coronanepal.live/' #credits to this site :)
page_content = requests.get(url).content #requesting information from the url
soup = BeautifulSoup(page_content, 'html.parser') #parsing the html content

In [39]:
#function to convert nepali numbers into english
def nep_to_eng(numbers):
  to_return = ''
  for i in numbers:
    conversion = {
        '०':'0',
        '१':'1',
        '२':'2',
        '३':'3',
        '४':'4',
        '५':'5',
        '६':'6',
        '७':'7',
        '८':'8',
        '९':'9',
    }
    to_return = to_return + (conversion.get(i, ''))
  return int(to_return) 


In [40]:
'''Not the good idea to scrap data but works fine for this page'''

test1 = soup.find_all('h1', attrs={'style': 'color:white;'}) #scrapping total tests in nepal
test2 = soup.find_all('h1', attrs={'style': 'color:#f38a03'}) #scrapping infected in nepal and world
test3 = soup.find_all('h1', attrs={'style': 'color:#e60000 !important;'}) #scrapping death in nepal and world
test4 = soup.find_all('h1', attrs={'style': 'color:#70a800 !important;'}) #scrapping recovered in nepal and world

list_of_values = []
for i in test1, test2, test3, test4:
  for j in i:
    j = "".join([k for k in j.string if ',' not in k]) #removing "," from the numbers
    returned = nep_to_eng(j)
    list_of_values.append(returned)

dict_keys = ['Test in Nepal', 'Infected in Nepal', 'Infected in World', 'Death in Nepal',
             'Death in World', 'Recovered in Nepal', 'Recovered in World']
dff = [i for i in zip(dict_keys, list_of_values)] #combining variables and values
df = pd.DataFrame(dff, columns=['Variable', 'Value']) #make dataframe for further manipulation
dfff = pd.pivot_table(df, values='Value', columns='Variable') #rows into columns
pd.set_option('display.max_columns', 20) #to fit while displaying
dfff = dfff[['Test in Nepal', 'Infected in Nepal', 'Death in Nepal', 'Recovered in Nepal', 
             'Infected in World', 'Death in World', 'Recovered in World']] #arranging in some order
print(dfff)

Variable  Test in Nepal  Infected in Nepal  Death in Nepal  \
Value            375416              19771              56   

Variable  Recovered in Nepal  Infected in World  Death in World  \
Value                  14399           17631323          679673   

Variable  Recovered in World  
Value               11028301  


In [41]:
t1 = []
t2 = []
t3 = []
t4 = []
t5 = []
t6 = []
select_next = soup.find('table') #Find the table that contents district data
for i in select_next.select('tr'): #For each unique district
  j = i.select('td') #Values of each district
  if len(j)==6:
      t1.append(j[0].find(text=True)) #filter text with text=True
      t2.append(str(j[1].find(text=True))) #district name should be string
      t3.append(int(j[2].find(text=True))) #number of confirmed cases should be numeric
      t4.append(j[3].find(text=True))
      t5.append(j[4].find(text=True))
      t6.append(j[5].find(text=True))
df = pd.DataFrame()
df['District'] = t2
df['Confirmed cases'] = t3
df['Total death'] = t5
df['Total recovered'] = t6
list_eng_recovered = []
list_eng_death = []

df['Total recovered'] = [nep_to_eng(i) for i in df['Total recovered']] #Convert to english numerals
df['Total death'] = [nep_to_eng(i) for i in df['Total death']]
df['District'] = [i.capitalize() for i in df['District']] #Capitalizing  the first letter
df['Active cases'] = df['Confirmed cases'] - df['Total recovered']

df.sort_values(['Active cases'], ascending=False, ignore_index=True, inplace=True)
print(df.head(40))


        District  Confirmed cases  Total death  Total recovered  Active cases
0       Rautahat             1501            2              370          1131
1        Kailali              979            4               34           945
2      Mahottari              782            0               83           699
3         Achham              646            1                7           639
4         Bajura              626            1                0           626
5           Doti              626            0               10           616
6           Dang              670            1               89           581
7          Palpa              563            1                1           562
8        Sarlahi              755            0              226           529
9        Dailekh              834            2              308           526
10    Kanchanpur              463            0                7           456
11       Surkhet              545            1              119 

In [58]:
#Replacing out other districts with  others

df_1 = df[:35]
df_2 = df[35:]
sum_cols = ['Confirmed cases', 'Total death', 'Total recovered', 'Active cases']
Others = dict(df_2[sum_cols].sum(axis=0))
data = Others.update(dict(District = 'Others'))
df_others = pd.DataFrame({'Confirmed cases': [Others['Confirmed cases']], 'Total death': [Others['Total death']], 'Total recovered': [Others['Total recovered']], 'Active cases': [Others['Active cases']], 'District': [Others['District']]})
df_plot = df_1.append(df_others, ignore_index=True)
print(df_plot)

        District  Confirmed cases  Total death  Total recovered  Active cases
0       Rautahat             1501            2              370          1131
1        Kailali              979            4               34           945
2      Mahottari              782            0               83           699
3         Achham              646            1                7           639
4         Bajura              626            1                0           626
5           Doti              626            0               10           616
6           Dang              670            1               89           581
7          Palpa              563            1                1           562
8        Sarlahi              755            0              226           529
9        Dailekh              834            2              308           526
10    Kanchanpur              463            0                7           456
11       Surkhet              545            1              119 

In [59]:
#Pie plot
try:
  import chart_studio
except:
  !pip install chart_studio
  import chart_studio
import chart_studio.plotly as py
import plotly.express as px
chart_studio.tools.set_credentials_file(username='raygtm', api_key='Gi1F46rnray1qGk83fFz4e9')
gbb = df_plot.groupby('District')[['Active cases']].sum().reset_index()
fig = px.pie(gbb, values='Active cases', names='District', title='District wise covid-19 active cases')
fig.update_traces(rotation = 310,pull=0.05, textinfo='percent+label')
#fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.update_layout(height=750)
fig.show()
py.iplot(fig, filename='COVID19 District') 